In [1]:
#!/usr/bin/env python3

# core openmm imports
from simtk.openmm import *
from simtk.openmm.app import *
from simtk.unit import *

from sys import stdout

# mdtraj provides extended trajectory output formats such as HDF5 and NetCDF
import mdtraj as mdt

# ngl view permits visualization of the system
import nglview as ng

Let us run a MD simulation of alanine dipeptide in vacuum using the AMBER 99 forcefield: first we need an input object (e.g. a pdb), and a forcefield definition (several are provided with the OpenMM installation)

In [2]:
# Input Files
print('Parsing PDB file...')
pdb = PDBFile('input/ala2.pdb')

print('Parsing FF file(s)...')
forcefield = ForceField('amber99sbildn.xml')


Parsing PDB file...
Parsing FF file(s)...


Define simulation parameters here: periodic boundaries if any, nonbonded interaction cutting scheme, rigid constraints for some of the bonds, etc.

In [3]:
# System Configuration
nonbondedMethod = CutoffNonPeriodic
nonbondedCutoff = 1.4*nanometers
constraints = HBonds
constraintTolerance = 0.00001

Create an Integrator : we use 'Langevin' at a temperature of 300 K ; for systems where a simulation box is defined it is also possible to add a barostat and then run in the NPT ensemble

In [4]:
# Integration Options
dt = 0.002*picoseconds
temperature = 300.00*kelvin
friction = 2/picosecond

integrator = LangevinIntegrator(temperature, friction, dt)
integrator.setConstraintTolerance(constraintTolerance)


An OpenMM platform object is used for providing a level of hardware abstraction : one can choose Reference, CPU, OpenCL or CUDA

In [5]:
platform = Platform.getPlatformByName('CPU')
platformProperties = {'CpuThreads':'1'}

#platform = Platform.getPlatformByName('OpenCL')
#platformProperties = {'OpenCLPrecision':'mixed'}

#platform = Platform.getPlatformByName('CUDA')
#platformProperties = {'CudaPrecision': 'mixed'}


A system object is created from the pdb, the forcefield and some of the simulation parameters above defined : it contains coordinates, velocities,
atom types, and all the data required for energy/force evaluations.

But the computation is performed within a simulation object (using the above defined platform which actually contains the harware dependent compiled code).

In [6]:
# Prepare the Simulation
print('Building system...')
topology = pdb.topology
positions = pdb.positions
system = forcefield.createSystem(topology, nonbondedMethod=nonbondedMethod,
                                 nonbondedCutoff=nonbondedCutoff,
                                 constraints=constraints)

simulation = Simulation(topology, system, integrator,
                        platform, platformProperties)

# set initial positions to what was one obtained from PDB parsing
simulation.context.setPositions(positions)


Building system...


Energy minimization is performed using the L-BFGS method (https://en.wikipedia.org/wiki/Limited-memory_BFGS)

In [7]:
# Minimize energy
print('Performing energy minimization...')
simulation.minimizeEnergy()

# after minimization of the energy, set initial velocities to random distribution for a given temperature
simulation.context.setVelocitiesToTemperature(temperature)


Performing energy minimization...


Perform some steps of temperature equilibration :

In [8]:
equilibrationSteps = 25000
printFreq = 250

# reporters objects are used for terminal IO and trajectory IO
#  here we use the StateDataReporter for regularly printing simulation time, energy, etc. to a terminal
simulation.reporters.append(StateDataReporter(stdout, printFreq, step=True, 
    time=True, potentialEnergy=True, kineticEnergy=True, temperature=True))

print('Equilibrating...')
simulation.step(equilibrationSteps)

Equilibrating...
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Temperature (K)"
250,0.5000000000000003,61.69791289546146,63.21198316112848,298.14294400719956
500,1.0000000000000007,111.23739585845304,48.729957426705525,229.8376390361001
750,1.500000000000001,85.08710300300032,77.65620845498577,366.269961032384
1000,2.0000000000000013,81.31684654038045,57.52660940514127,271.32755261106865
1250,2.4999999999999463,88.79200073782332,64.26787246355815,303.12310646104385
1500,2.999999999999891,65.51939790115708,68.56565182010557,323.3938292888379
1750,3.499999999999836,88.2968232537487,51.685363422931104,243.77698092014128
2000,3.999999999999781,79.67389740505163,61.7413243969083,291.2064975983448
2250,4.499999999999726,68.46983221550656,69.53919459548648,327.9855996249961
2500,4.999999999999671,73.53631167257123,65.57428315257629,309.2848674030081
2750,5.499999999999616,90.14644581770472,55.778951198850514,263.0846224468038
3000,5.999999999999561,69.8265462968

Now that the system is temperature equilirated we can run the simulation ; we use the mdtraj package which provides extra trajectory formats (e.g. HDF5)

In [9]:
# reste step and time counters
simulation.currentStep = 0
simulation.context.setTime(0.0)

# save the equilibration results to file : state is platform independent but less precise, checkpoint file
#  contains the binary representation but it is platform dependent
simulation.saveState('output/eq.state')
simulation.saveCheckpoint('output/eq.chk')

simsteps = 5e5 # 1 ns
svfreq=500

simulation.reporters = []

simulation.reporters.append(StateDataReporter(stdout, svfreq, step=True, 
    time=True, potentialEnergy=True, kineticEnergy=True, totalEnergy=True,
    temperature=True, progress=True,remainingTime=True, speed=True, totalSteps=simsteps))

hdf5rep = mdt.reporters.HDF5Reporter('output/sim_1ns.hdf5', svfreq,
                                     coordinates=True, time=True, cell=True, potentialEnergy=True,
                                     kineticEnergy=True, temperature=True, velocities=True)

# dcdrep = DCDReporter('output/sim_1ns.dcd', svfreq)

simulation.reporters.append(hdf5rep)
# simulation.reporters.append(dcdrep)

simulation.step(simsteps)

simulation.saveState('output/sim_1ns.state')
simulation.saveCheckpoint('output/sim_1ns.chk')

hdf5rep.close()


#"Progress (%)","Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)","Time Remaining"
0.1%,500,1.0000000000000007,67.27385802639705,61.954839624287715,129.22869765068475,292.21355441412294,0,--
0.2%,1000,2.0000000000000013,98.13560299778823,59.803892084306476,157.9394950820947,282.06848697745687,1.99e+03,0:43
0.3%,1500,2.999999999999891,100.61851531835615,65.9593000822561,166.57781540061225,311.10082183390693,2.18e+03,0:39
0.4%,2000,3.999999999999781,108.23932273014964,85.00164740479624,193.24097013494588,400.915145125712,2.26e+03,0:38
0.5%,2500,4.999999999999671,93.44725826809585,54.69430864399618,148.14156691209203,257.9688436287992,2.28e+03,0:37


/home/fhedin/bin/anaconda3/envs/openmm-tutorials/lib/python3.5/site-packages/mdtraj/formats/hdf5.py:198: UserWarning: The .h5 extension is recommended.
  warnings.warn('The .h5 extension is recommended.')


0.6%,3000,5.999999999999561,70.67214961713992,37.78762599996238,108.45977561710231,178.2275052809885,2.21e+03,0:38
0.7%,3500,6.999999999999451,84.03150992810856,93.54793655317752,177.57944648128608,441.22420805354704,2.17e+03,0:39
0.8%,4000,7.999999999999341,76.6771912568689,85.53517589769588,162.2123671545648,403.43156286219335,2.21e+03,0:38
0.9%,4500,8.999999999999675,94.12297619783577,77.99957999992697,172.12255619776272,367.889492617655,2.19e+03,0:39
1.0%,5000,10.000000000000009,100.64484171631855,60.300625463851134,160.9454671801697,284.4113584514713,2.21e+03,0:38
1.1%,5500,11.000000000000343,72.76607580604427,61.95498190811311,134.72105771415738,292.21422550394595,2.16e+03,0:39
1.2%,6000,12.000000000000677,87.38803436042221,67.71524088867804,155.10327524910025,319.3828173567265,2.13e+03,0:40
1.3%,6500,13.00000000000101,86.83957228135729,76.20280355420523,163.04237583556252,359.41489346001157,2.1e+03,0:40
1.4%,7000,14.000000000001345,62.8355023327164,59.49573872327135,122.33124105

7.7%,38500,77.00000000000195,89.53416118288095,66.26682056025221,155.80098174313315,312.55125980574775,2.16e+03,0:36
7.8%,39000,77.99999999999962,108.68552730454095,73.72190705752608,182.40743436206702,347.71360284535615,2.16e+03,0:36
7.9%,39500,78.99999999999729,82.43765561915406,53.94493155785539,136.38258717700944,254.43436362262628,2.16e+03,0:36
8.0%,40000,79.99999999999496,63.67544843153897,63.08451875639497,126.75996718793394,297.54175083174573,2.16e+03,0:36
8.1%,40500,80.99999999999262,62.95526600687009,54.06625916962761,117.0215251764977,255.0066122621117,2.15e+03,0:36
8.2%,41000,81.9999999999903,76.58320534594633,61.68289950004921,138.26610484599553,290.93093322143733,2.15e+03,0:36
8.3%,41500,82.99999999998796,92.73621911551953,69.09137860819482,161.82759772371435,325.873449837126,2.15e+03,0:36
8.4%,42000,83.99999999998563,90.65063539312891,65.93487977807123,156.58551517120014,310.9856420686294,2.15e+03,0:36
8.5%,42500,84.9999999999833,100.00953612597857,53.06736617739346,153.

14.7%,73500,146.9999999999738,84.25383308856922,46.67795217823969,130.9317852668089,220.15923859205196,2.16e+03,0:34
14.8%,74000,147.99999999997857,102.46100098366237,60.156417105175166,162.61741808883755,283.73119145692374,2.16e+03,0:34
14.9%,74500,148.99999999998334,86.59236191548712,80.29403713234699,166.88639904783412,378.7114312778344,2.17e+03,0:33
15.0%,75000,149.99999999998812,78.65977207724359,53.6409667632324,132.300738840476,253.00069623534566,2.17e+03,0:33
15.1%,75500,150.9999999999929,72.95725048276915,50.35328481137001,123.31053529413916,237.4941557493583,2.17e+03,0:33
15.2%,76000,151.99999999999767,73.35602270132603,67.29120806090742,140.64723076223345,317.38284220478454,2.17e+03,0:33
15.3%,76500,153.00000000000244,97.49887092709591,37.56343171534904,135.06230264244493,177.1700800792854,2.17e+03,0:33
15.4%,77000,154.00000000000722,88.73237855867166,44.82640303249205,133.55878159116372,211.42630085333093,2.17e+03,0:33
15.5%,77500,155.000000000012,85.33683691957611,51.59953

...

70.7%,353500,706.9999999971054,100.61184398886721,53.33284740801723,153.94469139688442,251.54743362475347,2.24e+03,0:11
70.8%,354000,707.9999999970818,95.05954533642361,55.55775398453053,150.61729932095415,262.04133309903176,2.24e+03,0:11
70.9%,354500,708.9999999970581,83.10283770843658,69.40210955521671,152.5049472636533,327.33903017025847,2.24e+03,0:11
71.0%,355000,709.9999999970345,89.72332799623388,51.69005113312774,141.4133791293616,243.79909077414024,2.24e+03,0:11
71.1%,355500,710.9999999970108,82.74457333635095,47.605487264661114,130.35006060101207,224.5340110245313,2.24e+03,0:11
71.2%,356000,711.9999999969872,79.7149999110837,44.60287191587792,124.3178718269616,210.3720035215311,2.24e+03,0:11
71.3%,356500,712.9999999969635,85.98313545846973,50.593821561593074,136.57695702006282,238.62866112740707,2.24e+03,0:11
71.4%,357000,713.9999999969399,76.85501558444611,47.71126173546317,124.56627731990928,225.03290238260408,2.24e+03,0:11
71.5%,357500,714.9999999969162,69.7107172111449,50.

77.7%,388500,776.9999999954501,68.25970902295677,53.01579421560598,121.27550323856275,250.05203405863924,2.25e+03,0:08
77.8%,389000,777.9999999954265,89.64630496151709,48.00471022428744,137.65101518580454,226.41696901043827,2.25e+03,0:08
77.9%,389500,778.9999999954028,68.92324526019638,90.0503885182425,158.97363377843888,424.72782215019566,2.25e+03,0:08
78.0%,390000,779.9999999953792,74.82877679219493,80.72435315443956,155.5531299466345,380.7410414761589,2.25e+03,0:08
78.1%,390500,780.9999999953556,80.70181294149066,59.63973291965364,140.3415458611443,281.294220862271,2.25e+03,0:08
78.2%,391000,781.9999999953319,79.88116195906933,70.0202635998497,149.90142555891902,330.25458917506074,2.25e+03,0:08
78.3%,391500,782.9999999953083,81.8678509574255,67.18634613577609,149.0541970932016,316.88825492070544,2.25e+03,0:08
78.4%,392000,783.9999999952846,78.63875728754576,69.7758037132454,148.41456100079114,329.1015800992643,2.25e+03,0:08
78.5%,392500,784.999999995261,98.36534061157501,50.11998179

84.7%,423500,846.9999999937949,74.60959866141903,65.465392923981,140.07499158540003,308.77128039460393,2.25e+03,0:05
84.8%,424000,847.9999999937712,93.87488683325589,72.99941823704472,166.8743050703006,344.3059428863017,2.25e+03,0:05
84.9%,424500,848.9999999937476,90.77449156350164,78.90243130471748,169.67692286821912,372.14784257837124,2.25e+03,0:05
85.0%,425000,849.9999999937239,79.37107101921242,48.37570323884344,127.74677425805586,228.16677884133514,2.25e+03,0:05
85.1%,425500,850.9999999937003,108.05687407293661,85.74800128273175,193.80487535566834,404.43536599699326,2.25e+03,0:05
85.2%,426000,851.9999999936766,64.88049725910842,62.45317408893219,127.33367134804061,294.56397749783855,2.25e+03,0:05
85.3%,426500,852.999999993653,86.85958865119427,72.95474404647692,159.8143326976712,344.09523450426803,2.25e+03,0:05
85.4%,427000,853.9999999936293,72.33496445733053,54.84427560778537,127.17924006511589,258.6761714146395,2.25e+03,0:05
85.5%,427500,854.9999999936057,87.34763256370209,75.85

91.7%,458500,916.9999999921396,91.9801950875187,62.95472989032874,154.93492497784743,296.92959420110014,2.25e+03,0:03
91.8%,459000,917.9999999921159,95.33709032063905,61.70323664008851,157.04032696072755,291.0268545088386,2.25e+03,0:03
91.9%,459500,918.9999999920923,87.09169729780547,44.354827317196865,131.44652461500232,209.2020869456289,2.25e+03,0:03
92.0%,460000,919.9999999920686,88.95424396405048,72.24477342028679,161.19901738433725,340.7466173265503,2.25e+03,0:03
92.1%,460500,920.999999992045,80.55694310344155,97.29754042067545,177.854483524117,458.9094297474626,2.25e+03,0:03
92.2%,461000,921.9999999920213,74.20087268831998,62.4569574931271,136.65783018144708,294.58182214071525,2.25e+03,0:02
92.3%,461500,922.9999999919977,66.27542303567033,71.63321339736231,137.90863643303265,337.86215940332636,2.25e+03,0:02
92.4%,462000,923.999999991974,82.51915757102867,65.52770652298335,148.04686409401202,309.0651860582008,2.25e+03,0:02
92.5%,462500,924.9999999919504,91.77528839205229,64.483533

98.7%,493500,986.9999999904843,86.46560875339468,49.74591479764566,136.21152355104033,234.62946024483844,2.24e+03,0:00
98.8%,494000,987.9999999904607,68.08086036013574,65.07206264398593,133.15292300412167,306.9161155701436,2.24e+03,0:00
98.9%,494500,988.999999990437,80.108156246587,72.41112745918792,152.5192837057749,341.5312356920119,2.24e+03,0:00
99.0%,495000,989.9999999904134,68.44721603672642,89.24033223911019,157.6875482758366,420.9071452501148,2.24e+03,0:00
99.1%,495500,990.9999999903897,72.04124298605382,70.73532619824913,142.77656918430296,333.627222931746,2.24e+03,0:00
99.2%,496000,991.9999999903661,74.16545601273832,71.03568879505289,145.2011448077912,335.04390034641364,2.24e+03,0:00
99.3%,496500,992.9999999903424,81.07189529983356,39.9058809829191,120.97777628275266,188.21837639741628,2.24e+03,0:00
99.4%,497000,993.9999999903188,84.91486075406891,58.03953047179516,142.95439122586407,273.74677423975413,2.24e+03,0:00
99.5%,497500,994.9999999902951,84.00517640163802,64.65651768

When simulation finished we can have a look at the trajectory using an external trajectory viewer (PyMOL, VMD,...) or using nglview within this jupyter notebook:

In [10]:
traj = mdt.load_hdf5("output/sim_1ns.hdf5")

view = ng.show_mdtraj(traj)
view.add_ball_and_stick("all")

view